# Plots of atmospheric ducting transform
Refer to:

ITU-R P.453-11 'The radio refractive index: its formula and refractivity data'

ESD-TR-81-102 'A Model to Calculate EM Fields in Tropospheric Duct Environments at Frequencies through SHF'

AD-A145 321 'A Radar Sea Clutter Model of Atmospheric Ducting Conditions'

AD-A268 101 'Forecasting the Ducting of Electromagnetic Waves on the Mesoscale'

In [ ]:
from torchsig.signals.signal_types import Signal
from torchsig.datasets.dataset_metadata import NarrowbandMetadata
from torchsig.signals.builders.tone import ToneSignalBuilder
import torchsig.transforms.functional as F
from torchsig.utils.dsp import (
    torchsig_float_data_type,
    torchsig_complex_data_type
)

import numpy as np
import scipy.signal as sp
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.signal import spectrogram

from scipy.constants import c

In [ ]:
def generate_tone_signal(num_iq_samples: int = 10, scale: float = 1.0) -> Signal:
    """Generate a scaled, high SNR baseband tone Signal.

        Args:
        num_iq_samples (int, optional): Length of sample. Defaults to 10.
        scale (int, optional): scale normalized signal data. Defaults to 1.0.

        Returns:
            signal: generated Signal.

    """
    sample_rate = 10e6
    md = NarrowbandMetadata(
        num_iq_samples_dataset = num_iq_samples,
        fft_size = 4,
        impairment_level = 0,
        sample_rate = sample_rate,
        num_signals_min = 1,
        num_signals_distribution = [1.0],
        snr_db_min = 100.0,
        snr_db_max = 100.0,
        signal_duration_min = 1.00*num_iq_samples/sample_rate,
        signal_duration_max = 1.00*num_iq_samples/sample_rate,
        signal_bandwidth_min = sample_rate/4,
        signal_bandwidth_max = sample_rate/4,
        signal_center_freq_min = 0.0,
        signal_center_freq_max = 0.0,         
        class_list = ['tone'],
        class_distribution = [1.0],
        seed = 42
    )

    builder = ToneSignalBuilder(
        dataset_metadata = md, 
        class_name = 'tone',
        seed = 42
    )
    signal = builder.build()

    # normalize, then scale data   
    signal.data = F.normalize(
        data = signal.data,
        norm_order = 2,
        flatten = False
    )
    signal.data = np.multiply(signal.data, scale).astype(torchsig_complex_data_type)

    return signal

In [ ]:
def ducting(
    data: np.ndarray,
    sampling_rate: float = 4.0,
    frequency: float = 3e9,
    am_rate: float = 0.5,
    am_amp: float = 0.1,
    duct_height: float = 1e2,
    refractive_gradient: float = -2e5
) -> np.ndarray:
    """Applies a narrowband atmospheric ducting model with a phase factor and time-varying amplitude response.
    Refer to:
        ITU-R P.453-11 'The radio refractive index: its formula and refractivity data'
        Bean, B. R., and E.J. Dutton, 1968:, Radio Meteorology, Dover Publications, Inc., New York, 258 pp.

    Args:
        data (np.ndarray): Complex valued IQ data samples.
        sampling_rate (float): Sampling rate. Default 4.0.
        frequency (float): Carrier frequency in Hz. Default 3.0e9 Hz.
        am_rate (float): Slow amplitude modulation rate for time variation in Hz. Default 0.5 Hz.
        am_amp (float): Amplitude modulation sinusoidal amplitude. Default 0.1 V.
        duct_height (float): Atmospheric duct height in meters. Default 1e2 m.
        refractive_gradient (float): Refractive index gradient in N-units/m. Default -1.57e5 N/m.
    
    Returns:
        np.ndarray: Data with ducting effects.
    
    """
    N = len(data)
    wavelength = c / frequency

    # phase modification factor
    phase_factor = np.exp(1j * 2 * np.pi * duct_height * refractive_gradient / wavelength)
    data *= phase_factor
    
    # Slow time-varying amplitude modulation
    t = np.arange(N) / sampling_rate
    data *= 1 + am_amp * np.sin(2 * np.pi * am_rate * t)

    return data.astype(torchsig_complex_data_type)

In [ ]:
# test cases
rng = np.random.default_rng(42)

sampling_rate = 4.0
frequency = 3e9
am_rate = 0.5
am_amp = 0.1
duct_height = 1e2
refractive_gradient = -2e5

N = 10000
tone_bb_data = generate_tone_signal(num_iq_samples = N, scale = 1.0).data 
tone_data = tone_bb_data * np.exp(2j * np.pi * 0.2 * np.arange(N) / sampling_rate) # f0 = 0.2

data_d = ducting(
    data = tone_data,
    sampling_rate = sampling_rate,
    frequency = frequency, 
    am_rate = am_rate,
    am_amp = am_amp,
    duct_height = duct_height,
    refractive_gradient = refractive_gradient
)

In [ ]:
# plots
plt.style.use('dark_background')
fig, ax = plt.subplots()

spectrum, freqs, _ = ax.magnitude_spectrum(data_d, Fs=sampling_rate, scale='linear', sides='twosided', color='white');
ax.set_yscale('log')

ax.set_xlabel('Frequency (Hz)');
ax.set_ylabel('Magnitude [log]');
plt.ylim([1e-10, None]);